In [8]:
import requests
import json
openrouter_key='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'
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
  },
  data=json.dumps({
    "model": "deepseek/deepseek-chat-v3.1:free",
    "messages": [
      {
        "role": "user",
        "content": "解释一下多因子策略在投资组合中的作用?用中文回答"
      }
    ],
    
  })
)

In [7]:
response.json()["choices"][0]["message"]["content"]

KeyError: 'choices'

In [5]:
import requests
import json
openrouter_key='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'
response = requests.get(
  url="https://openrouter.ai/api/v1/key",
  headers={
    "Authorization": f"Bearer {openrouter_key}"
  }
)
print(json.dumps(response.json(), indent=2))

{
  "error": {
    "message": "User not found.",
    "code": 401
  }
}


In [4]:
import pandas as pd 
df=pd.read_parquet(r'D:\projects\q\data\minute_data\000002\2025-09.parquet')

In [5]:
df

,date,open,high,low,close,volume
0,2025-09-03 13:53:00,6.60,6.60,6.59,6.60,224300
1,2025-09-03 13:54:00,6.60,6.60,6.59,6.60,695600
2,2025-09-03 13:55:00,6.60,6.61,6.60,6.61,339600
3,2025-09-03 13:56:00,6.61,6.61,6.60,6.61,406800
4,2025-09-03 13:57:00,6.61,6.61,6.59,6.60,446900
...,...,...,...,...,...,...
1965,2025-09-15 14:54:00,NaN,NaN,NaN,NaN,649200
1966,2025-09-15 14:55:00,NaN,NaN,NaN,NaN,1186100
1967,2025-09-15 14:56:00,NaN,NaN,NaN,NaN,2255700
1968,2025-09-15 14:57:00,NaN,NaN,NaN,NaN,2261800
